## Summarization

we try to summarize on the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset .

In [ ]:
!pip install sentencepiece
!pip install 'transformers[torch]'
!pip install datasets
!pip install zstandard
!pip install sacrebleu
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3c0cc54c77f51f4d84993f05c3572c030e87f167d5c5d1fde0fa772fcf4648ca
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from datasets import load_metric
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
ds = load_dataset("cnn_dailymail",  version="3.0.0", split="train")

train_testvalid = ds.train_test_split(train_size=1000, test_size=200)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 100
    })
})

In [ ]:
ds["train"][1]

{'article': "By . David Wilkes . Paralysed by grief, Sarah Ambrose could not bear to look at her son’s\xa0 belongings after he was killed at the Battle of the Somme. So 18-year-old Edward Ambrose’s possessions were packed in a leather suitcase and stored away in the attic of the family home. And there they stayed for more than 90 years – until curiosity finally got the better of Edward’s nephew John, 82, who retrieved the case after reading about an appeal for untold stories for a First World War exhibition. The 'time capsule' includes a locket containing photos of Private Ambrose and his sweetheart 'Gladys', who he left at home to go and fight on the Western Front . Amazingly, Private Ambrose's nephew, John, also found two fragments of the German shell which killed his uncle in the Battle of the Somme on July 6, 1916. Records show in was struck in the leg by a shell when his line came under heavy bombardment . Touchingly, the case included a guide book to Paris, which Private Ambrose 

In [ ]:
sample_text = ds["train"][1]["article"][:2000]
sample_text


"By . David Wilkes . Paralysed by grief, Sarah Ambrose could not bear to look at her son’s\xa0 belongings after he was killed at the Battle of the Somme. So 18-year-old Edward Ambrose’s possessions were packed in a leather suitcase and stored away in the attic of the family home. And there they stayed for more than 90 years – until curiosity finally got the better of Edward’s nephew John, 82, who retrieved the case after reading about an appeal for untold stories for a First World War exhibition. The 'time capsule' includes a locket containing photos of Private Ambrose and his sweetheart 'Gladys', who he left at home to go and fight on the Western Front . Amazingly, Private Ambrose's nephew, John, also found two fragments of the German shell which killed his uncle in the Battle of the Somme on July 6, 1916. Records show in was struck in the leg by a shell when his line came under heavy bombardment . Touchingly, the case included a guide book to Paris, which Private Ambrose sadly never 

## Todo
Implement a baseline summary. An easy way to generate a baseline summary of a newspaper article is to take the first few sentences only. Write a function to generate a baseline summary given the article text.

HINT: We have imported the sent_tokenize from nltk.

In [ ]:
def get_baseline_summary(text):
    #TODO: Write a function to generate a baseline summary

In [ ]:
summaries = {}

In [ ]:
summaries["baseline"] = get_baseline_summary(sample_text)

### GPT2
Can implement a summarization by using the text-generation pipeline but appending a "TL:DR" at the end of the prompt

In [ ]:
pipe = pipeline("text-generation", model="gpt2") #If gpt2-xl is too large use a small version
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))
summaries["gpt2"]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### T5

In [ ]:
pipe = pipeline("summarization", model="t5-small") #If t5-large is too large use t5-small version
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["t5"]

Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors


'18-year-old was killed at the Battle of the Somme on July 6, 1916 .\nhis nephew found two fragments of the german shell which killed his uncle .\nthe case contains poignant reminders of his final days before his death .'

## BART

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
summaries["bart"]

"Private Edward Ambrose, 18, was killed at the Battle of the Somme in 1916.\nHis mother, Sarah, could not bear to look at her son's belongings.\nThey were packed in a leather suitcase and stored away in the attic.\nBut curiosity finally got the better of Private Ambrose's nephew John, 82.\nHe retrieved the case after reading about an appeal for untold stories."

In [ ]:
print("GROUND TRUTH")
print(ds["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Private Edward Ambrose, from Hertfordshire, was killed on the Somme, just days he arrived at the front in 1916 .
After a telegram telling of his death, his last belongings were sent back from the trenches to his heartbroken parents .
But the family left the case unopened, finding its contents too painful to look at, and it was placed in an attic for years .
After visiting a local historical exhibition, Private Ambrose's nephew has now opened the package for the first time .
The case includes black and white photos of his family, letters from his parents, a half-smoked pipe and cigarettes .
The items, including a locket with photos of Private Ambrose and his sweetheart, Gladys, will go display later this year .

BASELINE
By .
David Wilkes .
Paralysed by grief, Sarah Ambrose could not bear to look at her son’s  belongings after he was killed at the Battle of the Somme.

GPT2
The diary of Corporal John Williams and his partner Lieutenant Colonel George Hernan.
The 'time capsu

## Evaluation

How can we evaluate the performance of a summarization model?

**BLEU (Bilingual Evaluation Understudy) Score**  is a precision based metric that measures how many words or n-grams present in the machine generated summaries also appear in the human reference summaries divided by the length of the generation. It penalises the appearance of too short summaries with a brevity penalty. [Sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu).

**ROUGE (Recall Oriented Understudy for Gisting Evaluation) score** is the recall based metric that measures how many words/ngrams present in the human reference summaries were found in the machine generated summaries. Some sub-forms:

1. ROUGE-N: Overlap of n-grams
2. ROUGE-1: Overlap of unigrams.
3. ROUGE-2: Oerlap of bigrams.
4. ROUGE-L: Overlap of the Longest Common Subsequence

To Do:
1. How do you interpret a Bleu/ Rouge Score of 0?

In [ ]:
bleu_metric = load_metric("sacrebleu")

In [ ]:
rouge_metric = load_metric("rouge")

In [ ]:
reference = ds["train"][1]["highlights"]
reference

"Private Edward Ambrose, from Hertfordshire, was killed on the Somme, just days he arrived at the front in 1916 .\nAfter a telegram telling of his death, his last belongings were sent back from the trenches to his heartbroken parents .\nBut the family left the case unopened, finding its contents too painful to look at, and it was placed in an attic for years .\nAfter visiting a local historical exhibition, Private Ambrose's nephew has now opened the package for the first time .\nThe case includes black and white photos of his family, letters from his parents, a half-smoked pipe and cigarettes .\nThe items, including a locket with photos of Private Ambrose and his sweetheart, Gladys, will go display later this year ."

In [ ]:
bleu_scores = []
for model_name in summaries:
  bleu_metric.add(prediction=summaries[model_name], reference=[reference])
  results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
  bleu_scores.append(results["score"])
  #results["precisions"] = [np.round(p, 2) for p in results["precisions"]]

In [ ]:
records

[{'rouge1': 0.2,
  'rouge2': 0.06756756756756757,
  'rougeL': 0.10666666666666666,
  'rougeLsum': 0.16000000000000003},
 {'rouge1': 0.34285714285714286,
  'rouge2': 0.09248554913294797,
  'rougeL': 0.16,
  'rougeLsum': 0.30857142857142855},
 {'rouge1': 0.271604938271605,
  'rouge2': 0.075,
  'rougeL': 0.1728395061728395,
  'rougeLsum': 0.23456790123456792},
 {'rouge1': 0.3850267379679144,
  'rouge2': 0.14054054054054055,
  'rougeL': 0.23529411764705885,
  'rougeLsum': 0.37433155080213903}]

In [ ]:

records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    rouge_score = rouge_metric.compute()
    result_dict = dict((rn, rouge_score[rn].mid.fmeasure) for rn in rouge_names)
    bleu_metric.add(prediction=summaries[model_name], reference=[reference])
    bleu_results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
    result_dict["bleu"] = bleu_results["score"]
    records.append(result_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum,bleu
baseline,0.200000,0.067568,0.106667,0.160000,0.000000
gpt2,0.342857,0.092486,0.160000,0.308571,3.738187
t5,0.271605,0.075000,0.172840,0.234568,0.000000
bart,0.385027,0.140541,0.235294,0.374332,4.575374


## Finetuning a model for Summarization

#Discussion
How would you fine tune a model for summarization?

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
def convert_data_to_features(batch, tokenizer):
    input_encodings = tokenizer(batch["article"], max_length=1024,
                                truncation=True)

    with tokenizer.as_target_tokenizer(): #as_target_tokenizer helps to differentiate between
        target_encodings = tokenizer(batch["highlights"], max_length=128,
                                     truncation=True)

    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

In [ ]:
dataset_cnn = ds.map(convert_data_to_features, batched=True, fn_kwargs={"tokenizer": tokenizer})
columns = ["input_ids", "labels", "attention_mask"]
dataset_cnn.set_format(type="torch", columns=columns)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='output_folder', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=False,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_cnn["train"],
                  eval_dataset=dataset_cnn["valid"])

In [ ]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=62, training_loss=3.6997735961791007, metrics={'train_runtime': 138.9662, 'train_samples_per_second': 7.196, 'train_steps_per_second': 0.446, 'total_flos': 449552617113600.0, 'train_loss': 3.6997735961791007, 'epoch': 0.99})

In [ ]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
rouge_score = evaluate_summaries(ds["test"], rouge_metric, trainer.model, tokenizer,batch_size=2)

100%|██████████| 50/50 [01:20<00:00,  1.60s/it]


In [ ]:
rouge_dict = dict((rn, rouge_score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"cnn"])

,rouge1,rouge2,rougeL,rougeLsum
cnn,0.250032,0.104014,0.189566,0.223711


## Todo
How would you calculate the BLEU score?